In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from scipy.spatial import distance
from scipy.sparse import csr_matrix
from numpy import transpose

In [2]:
def TSCAN(path, X):
    """function that calculate pseudo time"""
    #project value of the first cluster
    v12 = np.array(path[1]).reshape(-1,1) -np.array(path[0]).reshape(-1,1)
    x1 = data.loc[labels == 0].values
    proj_c1 = (x1 - x1.mean(axis = 0))@v12/(v12.T@v12)
    
    #project value of the last cluster
    v_last = np.array(path[-1]).reshape(-1,1) -np.array(path[-2]).reshape(-1,1)
    x_last = data.loc[labels == len(path)-1].values
    proj_last = (x_last - x_last.mean(axis = 0))@v_last/(v_last.T@v_last)

    #project value of internediate clusters
    lists = []
    for i in range(1, len(path)-1):
        project_value_list_minus = []
        project_value_list_add = []

        v_minus = np.array(path[i]).reshape(-1,1)-np.array(path[i-1]).reshape(-1,1)
        v_add = np.array(path[i+1]).reshape(-1,1)-np.array(path[i]).reshape(-1,1)
        for item in X[i]:
            d1 = np.linalg.norm(np.array(item) - path[i-1])
            d2 = np.linalg.norm(np.array(item) - path[i+1])
            xi = np.array(item).reshape(-1,1) - np.array(path[i]).reshape(-1,1)
            if d1 < d2:
                project_value_minus = (xi.T@(v_minus))/(v_minus.T@v_minus)
                project_value_list_minus.append(project_value_minus)
                
            else:
                project_value_add = (xi.T@(v_add))/(v_add.T@v_add)
                project_value_list_add.append(project_value_add)
                
            combine = (project_value_list_minus, project_value_list_add)
        lists.append(combine)
        flat_lists = list(np.concatenate(lists).flat)
        
        #remove empty in the list
        remove_empty = [x for x in flat_lists if x]
        project_value_int = list(np.concatenate(remove_empty).flat)
        

    #calculate order of by each cluster
    order_c1 = np.argsort(proj_c1.reshape(1, -1))
    order_c1_list = order_c1[0].tolist()
    
    cell_order_int = np.argsort(np.array(project_value_int, dtype=object).reshape(1, -1))
    order_int = cell_order_int[0] + max(order_c1[0])+1
    order_int_list = order_int.tolist()
    
    
    cell_order_last = np.argsort(proj_last.reshape(1, -1))
    order_last = cell_order_last[0]+max(order_int)+1
    order_last_list = order_last.tolist()
    
    
    
    orders_combine = order_c1_list + order_int_list + order_last_list
    orders = np.transpose(orders_combine)
        
    return orders
        


In [3]:
data = pd.read_csv("matrixnby2.csv")
matrix = np.array(data)
kmeans = KMeans(n_clusters=4)
kmeans.fit(matrix)
y_kmeans = kmeans.predict(matrix)

In [4]:
path = kmeans.cluster_centers_
path

array([[224.63789116, 194.72322871],
       [441.67149244,  68.28088238],
       [193.58825651,  84.23011483],
       [343.50471396,  47.17283803]])

In [5]:
labels = kmeans.labels_
labels

array([2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2,
       2, 2, 2, 1, 1, 3, 1, 1, 1, 1, 3, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1,
       1, 1, 3, 3, 3, 3, 1, 1, 3, 1, 1], dtype=int32)

In [6]:
c1_points_position = list(np.where(labels == 0)[0])
c2_points_position = list(np.where(labels == 1)[0])
c3_points_position = list(np.where(labels == 2)[0])
c4_points_position = list(np.where(labels == 3)[0])


x1 = [matrix[ele] for ele in c1_points_position ]
x2 = [matrix[ele] for ele in c2_points_position ]
x3 = [matrix[ele] for ele in c3_points_position ]
x4 = [matrix[ele] for ele in c4_points_position ]


In [7]:
X=(x1,x2,x3,x4)


In [8]:
TSCAN(path, X)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


array([ 7,  3,  4,  2,  1,  8,  0,  5,  6,  9, 34, 32, 29, 16, 31, 27, 33,
       21, 14, 35, 15, 13, 12, 20, 25, 10, 17, 44, 11, 22, 45, 24, 37, 26,
       36, 19, 43, 23, 39, 41, 30, 28, 40, 18, 42, 38, 51, 47, 46, 49, 53,
       48, 50, 54, 52])